In [ ]:
from pdf2image import convert_from_path


# Store Pdf with convert_from_path function
images = convert_from_path('./input/newspaper.pdf')
import os

if not os.path.exists('images'):
  os.makedirs('images')
for i in range(len(images)):

      # Save pages as images in the pdf
    images[i].save('images/page'+ str(i) +'.jpg', 'JPEG')

In [ ]:
import cv2
image = cv2.imread('./images/page15.jpg') # reading the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
(thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
cv2.imshow('binary', binary)
cv2.imwrite('binary.png', binary)

In [ ]:
(contours, _) = cv2.findContours(~binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
# find contours
for contour in contours:
    """
    draw a rectangle around those contours on main image
    """
    [x,y,w,h] = cv2.boundingRect(contour)
    cv2.rectangle(image, (x,y), (x+w,y+h), (0, 255, 0), 1)
cv2.imshow('contours.png', image)
cv2.imwrite('contours.png', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
mask = np.ones(image.shape[:2], dtype="uint8") * 255 # create blank image of same dimension of the original image
(contours, _) = cv2.findContours(~binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
heights = [cv2.boundingRect(contour)[3] for contour in contours] # collecting heights of each contour
avgheight = sum(heights)/len(heights) # average height
# finding the larger contours
# Applying Height heuristic
for c in contours:
    [x,y,w,h] = cv2.boundingRect(c)
    if h > 2*avgheight:
        cv2.drawContours(mask, [c], -1, 0, -1)
cv2.imshow('filter.png', mask)
cv2.imwrite('filter.png', ma Processing binary.pngsk)

In [ ]:
from pythonRLSA import rlsa
import math
x, y = mask.shape
value = max(math.ceil(x/100),math.ceil(y/100))+20 #heuristic
mask = rlsa.rlsa(mask, True, False, value) #rlsa application
cv2.imshow('rlsah.png', mask)
cv2.imwrite('rlsah.png', mask)

In [ ]:
(contours, _) = cv2.findContours(~mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # find contours
mask2 = np.ones(image.shape, dtype="uint8") * 255 # blank 3 layer image
for contour in contours:
    [x, y, w, h] = cv2.boundingRect(contour)
    if w > 0.60*image.shape[1]: # width heuristic applied
        title = image[y: y+h, x: x+w]
        mask2[y: y+h, x: x+w] = title # copied title contour onto the blank image
        image[y: y+h, x: x+w] = 255 # nullified the title contour on original image
cv2.imshow('title', mask2)
cv2.imwrite('title.png', mask2)
cv2.imshow('content', image)
cv2.imshow('content.png', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import pytesseract
title = pytesseract.image_to_string(Image.fromarray(mask2))
content = pytesseract.image_to_string(Image.fromarray(image))

In [ ]:
minLineLength = 100
maxLineGap = 50
def lines_extraction(gray):
    """
    this function extracts the lines from the binary image. Cleaning process.
    """
    edges = cv2.Canny(gray, 75, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength, maxLineGap)
    return lines
mask = np.ones(image.shape[:2], dtype="uint8") * 255 # create a white image
lines = lines_extraction(gray) # extract lines
try:
    for line in lines: # write lines to mask
        x1, y1, x2, y2 = line[0]
        cv2.line(mask, (x1, y1), (x2, y2), (0, 255, 0), 3)
except TypeError:
    pass
(contours, _) = cv2.findContours(~binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # find contours
areas = [cv2.contourArea(c) for c in contours] # find area of contour
avgArea = sum(areas)/len(areas) # finding average area
for c in contours:# average area heuristics
    if cv2.contourArea(c)>60*avgArea:
        cv2.drawContours(mask, [c], -1, 0, -1)
binary = cv2.bitwise_and(binary, binary, mask=mask) # subtracting the noise
cv2.imwrite('noise.png', mask)
cv2.imshow('mask', mask)
cv2.imshow('binary_noise_removal', ~binary)
cv2.imwrite('binary_noise_removal.png', ~binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow('binary_noise_removal', ~binary)